In [ ]:
import pandas as pd
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif #importance
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier 
from sklearn import svm #Support Vector Machine
from sklearn.neural_network import MLPClassifier
import keras
from keras import layers
import tensorflow as tf
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score




Chest_window30Location=["Chest_30Processed\Process30_chest_S2.csv","Chest_30Processed\Process30_chest_S3.csv","Chest_30Processed\Process30_chest_S4.csv","Chest_30Processed\Process30_chest_S5.csv","Chest_30Processed\Process30_chest_S6.csv",
                "Chest_30Processed\Process30_chest_S7.csv","Chest_30Processed\Process30_chest_S8.csv","Chest_30Processed\Process30_chest_S9.csv","Chest_30Processed\Process30_chest_S10.csv","Chest_30Processed\Process30_chest_S11.csv",
                "Chest_30Processed\Process30_chest_S13.csv","Chest_30Processed\Process30_chest_S14.csv","Chest_30Processed\Process30_chest_S15.csv","Chest_30Processed\Process30_chest_S16.csv","Chest_30Processed\Process30_chest_S17.csv"]

Chest_window60Location=["Chest_60Processed\Process60_chest_S2.csv","Chest_60Processed\Process60_chest_S3.csv","Chest_60Processed\Process60_chest_S4.csv","Chest_60Processed\Process60_chest_S5.csv","Chest_60Processed\Process60_chest_S6.csv",
                "Chest_60Processed\Process60_chest_S7.csv","Chest_60Processed\Process60_chest_S8.csv","Chest_60Processed\Process60_chest_S9.csv","Chest_60Processed\Process60_chest_S10.csv","Chest_60Processed\Process60_chest_S11.csv",
                "Chest_60Processed\Process60_chest_S13.csv","Chest_60Processed\Process60_chest_S14.csv","Chest_60Processed\Process60_chest_S15.csv","Chest_60Processed\Process60_chest_S16.csv","Chest_60Processed\Process60_chest_S17.csv"]

Wrist_window30Location=["Wrist_30Processed\Process30_wrist_S2.csv","Wrist_30Processed\Process30_wrist_S3.csv","Wrist_30Processed\Process30_wrist_S4.csv","Wrist_30Processed\Process30_wrist_S5.csv","Wrist_30Processed\Process30_wrist_S6.csv",
                "Wrist_30Processed\Process30_wrist_S7.csv","Wrist_30Processed\Process30_wrist_S8.csv","Wrist_30Processed\Process30_wrist_S9.csv","Wrist_30Processed\Process30_wrist_S10.csv","Wrist_30Processed\Process30_wrist_S11.csv",
                "Wrist_30Processed\Process30_wrist_S13.csv","Wrist_30Processed\Process30_wrist_S14.csv","Wrist_30Processed\Process30_wrist_S15.csv","Wrist_30Processed\Process30_wrist_S16.csv","Wrist_30Processed\Process30_wrist_S17.csv"]

Wrist_window60Location=["Wrist_60Processed\Process60_wrist_S2.csv","Wrist_60Processed\Process60_wrist_S3.csv","Wrist_60Processed\Process60_wrist_S4.csv","Wrist_60Processed\Process60_wrist_S5.csv","Wrist_60Processed\Process60_wrist_S6.csv",
                "Wrist_60Processed\Process60_wrist_S7.csv","Wrist_60Processed\Process60_wrist_S8.csv","Wrist_60Processed\Process60_wrist_S9.csv","Wrist_60Processed\Process60_wrist_S10.csv","Wrist_60Processed\Process60_wrist_S11.csv",
                "Wrist_60Processed\Process60_wrist_S13.csv","Wrist_60Processed\Process60_wrist_S14.csv","Wrist_60Processed\Process60_wrist_S15.csv","Wrist_60Processed\Process60_wrist_S16.csv","Wrist_60Processed\Process60_wrist_S17.csv"]

Combine60_Location=["Combine_Chest_Wrist\Combined_S2.csv","Combine_Chest_Wrist\Combined_S3.csv","Combine_Chest_Wrist\Combined_S4.csv","Combine_Chest_Wrist\Combined_S5.csv","Combine_Chest_Wrist\Combined_S6.csv",
                "Combine_Chest_Wrist\Combined_S7.csv","Combine_Chest_Wrist\Combined_S8.csv","Combine_Chest_Wrist\Combined_S9.csv","Combine_Chest_Wrist\Combined_S10.csv","Combine_Chest_Wrist\Combined_S11.csv",
                "Combine_Chest_Wrist\Combined_S13.csv","Combine_Chest_Wrist\Combined_S14.csv","Combine_Chest_Wrist\Combined_S15.csv","Combine_Chest_Wrist\Combined_S16.csv","Combine_Chest_Wrist\Combined_S17.csv"]



In [ ]:
# Loading function for Random splitting and chest wrist approach

def dataset_loading(window,signal_source,chest_wrist):
    frame=[]
    if window==30 and signal_source=="Chest":
        StoringLocation=Chest_window30Location
    elif window==60 and signal_source=="Chest":
        StoringLocation=Chest_window60Location
    elif window==30 and signal_source=="Wrist":
        StoringLocation=Wrist_window30Location
    elif window==60 and signal_source=="Wrist":
        StoringLocation=Wrist_window60Location
    else:
        StoringLocation=Combine60_Location

    for filepath in StoringLocation:
      file = pd.read_csv(filepath)
      frame.append(file)

    dataset=pd.concat(frame,ignore_index=True,axis=0)

    #remove null value identified from chest signals 30 seocnds csv file
    if window==30 and signal_source=="Chest":
        drop_col=['ecg_hrv_energy_LF', 'ecg_hrv_relative_power_LF', 'ecg_hrv_ratio_LFHF', 'ecg_hrv_norm_LF']
        dataset.drop(columns=drop_col,axis=1, inplace=True)
        dataset.dropna(inplace=True)

    #filtering down to the common signals for chest-wrist approach
    if(chest_wrist):
        dataset=dataset[[ "eda_mean","eda_std","eda_min","eda_max","eda_range",
        "eda_edl_mean","eda_edl_std","eda_edl_slope","eda_edr_std",
        "eda_edr_num","eda_edr_mean_amp","eda_edr_sum_amp","temp_mean",
        "temp_std","temp_min","temp_max","temp_range","temp_slope","label"]]

    labels=dataset["label"]

    dataset=dataset.drop("label",axis=1)
    Unique_class=np.sort(labels.unique())
    return dataset, labels,Unique_class


In [ ]:
def standardise_smote(training_x,training_y,valid_x,test_x):
    standardiser=StandardScaler()
    standardiser.fit(training_x)
    training_x=standardiser.transform(training_x)
    if(valid_x is not None):
      print("valiiiii")
      valid_x=standardiser.transform(valid_x)
    test_x=standardiser.transform(test_x)

    resampling=SMOTE(sampling_strategy="auto")
    X_resample_train, Y_resample_train = resampling.fit_resample(training_x, training_y)

    return X_resample_train,Y_resample_train,valid_x,test_x

In [ ]:
#conduct random splitting approach + mutual information
def dataset_handling(dataset, label,comp_num,sequence):

    #mutual information
    mutual_info= mutual_info_classif(dataset,label)
    mutual_info=pd.Series(mutual_info)
    mutual_info.index=dataset.columns.values
    mutual_info=mutual_info.sort_values(ascending=False)
    # Nair, M. 2023. Feature Selection — Mutual Information. Available at:https://medium.com/@miramnair/feature-selection-mutual-information-a0def943e1ed [Accessed:3 May 2025]

    X_train,x,Y_train,y=train_test_split(dataset,label, test_size=0.3,random_state=16, stratify=label)
    X_valid,X_test,Y_valid,Y_test=train_test_split(x,y,  test_size=0.5, random_state=16,stratify=y)
    
    X_resample_train,Y_resample_train,X_valid,X_test=standardise_smote(X_train,Y_train,X_valid,X_test)

    if (comp_num is not None):
        pca_transformer=PCA(n_components=comp_num)
        pca_transformer.fit(X_resample_train)
        X_resample_train=pca_transformer.transform(X_resample_train)
        X_valid=pca_transformer.transform(X_valid)
        X_test=pca_transformer.transform(X_test)

    Y_valid=Y_valid.reset_index(drop=True)
    Y_test=Y_test.reset_index(drop=True)

    #Print out to find the class distribution before and after balancing 
    # print(f"Y_train is {len(Y_train)}, distribution is {Counter(Y_train)}.")
    # print(f"After resample(SMOTE) is {len(Y_resample_train)}, distribution is {Counter(Y_resample_train)}.")
    # print(f"Y_valid is  {len(Y_valid)}, distribution is {Counter(Y_valid)}.")
    # print(f"Y_test is {len(Y_test)}, distribution is {Counter(Y_test)}")

    return X_resample_train,Y_resample_train,X_valid,Y_valid,X_test,Y_test,mutual_info

In [ ]:
# Loading and handling function for subject-wise splitting and average individual approach
def individual_Load_handling(window, target_num, comp_num,signal_source, mode):
    frame=[]
    if window==30 and signal_source=="Chest":
        StoringLocation=Chest_window30Location
    elif window==60 and signal_source=="Chest":
        StoringLocation=Chest_window60Location
    elif window==30 and signal_source=="Wrist":
        StoringLocation=Wrist_window30Location
    elif window==60 and signal_source=="Wrist":
        StoringLocation=Wrist_window60Location
    else:
        StoringLocation=Combine60_Location

    #randomly select two subject as valid set, another two subject as test set
    random.seed(10)
    indexpool=random.sample(range(15),k=4)
    valid_index=indexpool[:2]
    test_index=indexpool[2:]

    if(mode=="average"):
        valid_num=(target_num+1)%15 
        for filepath_index in range(len(StoringLocation)):#   ratio of 13:1:1 for each iteration
            if(filepath_index==target_num):
                file = pd.read_csv(StoringLocation[filepath_index])
                test_set=pd.DataFrame(data=file)
            elif (filepath_index==valid_num):
                file = pd.read_csv(StoringLocation[filepath_index])
                valid_set=pd.DataFrame(data=file)
            else:
                #The rest merge as one
                file = pd.read_csv(StoringLocation[filepath_index])
                frame.append(file)
        dataset=pd.concat(frame,ignore_index=True,axis=0)

    else:
        valid=[]
        test=[]
        #ratio of 11:2:2 on approach
        for filepath_index in range(len(StoringLocation)):
            if (filepath_index in valid_index):
                file = pd.read_csv(StoringLocation[filepath_index])
                valid.append(file)
            elif(filepath_index in test_index ):
                file = pd.read_csv(StoringLocation[filepath_index])
                test.append(file)
            else:
                file = pd.read_csv(StoringLocation[filepath_index])
                frame.append(file)
        dataset=pd.concat(frame,ignore_index=True,axis=0)
        valid_set=pd.concat(valid,ignore_index=True,axis=0)
        test_set=pd.concat(test,ignore_index=True,axis=0)


    if window==30 and signal_source=="Chest":#Remove null value found in 30 sec window of chest signals csv
        drop_col=['ecg_hrv_energy_LF', 'ecg_hrv_relative_power_LF', 'ecg_hrv_ratio_LFHF', 'ecg_hrv_norm_LF']
        dataset.drop(columns=drop_col,axis=1, inplace=True)
        dataset.dropna(inplace=True)
        valid_set.drop(columns=drop_col,axis=1, inplace=True)
        valid_set.dropna(inplace=True)
        test_set.drop(columns=drop_col,axis=1, inplace=True)
        test_set.dropna(inplace=True)

   #Seperate data and label for each
    Y_train=dataset["label"]
    X_train=dataset.drop("label",axis=1)
    Unique_class=np.sort(Y_train.unique())
    Y_valid=valid_set["label"]
    X_valid=valid_set.drop("label",axis=1)
    Y_test=test_set["label"]
    X_test=test_set.drop("label",axis=1)

    X_resample_train,Y_resample_train,X_valid,X_test=standardise_smote(X_train,Y_train,X_valid,X_test)

    #reduce number of features
    if (comp_num is not None):
        pca_transformer=PCA(n_components=comp_num)
        pca_transformer.fit(X_resample_train)
        X_resample_train=pca_transformer.transform(X_resample_train)
        X_test=pca_transformer.transform(X_test)
        X_valid=pca_transformer.transform(X_valid)

    return X_resample_train,Y_resample_train,X_valid,Y_valid,X_test,Y_test, Unique_class

In [ ]:
def KNN_classify(training_x,training_y,target_x,k):
    kNN=KNeighborsClassifier(n_neighbors=k,weights="distance")
    kNN.fit(training_x,training_y)
    KNN_classification=kNN.predict(target_x)
    return KNN_classification

def NaiveBayes_classify(training_x, training_y, target_x):
    NBclassifer=GaussianNB()
    NBclassifer.fit(training_x,training_y)
    GaussianNB_classification=NBclassifer.predict(target_x)
    return GaussianNB_classification

def RandomForest_classify(training_x,training_y,target_x, tree_number):
    RanTree=RandomForestClassifier(n_estimators=tree_number, max_depth=11, min_samples_split=5)
    RanTree.fit(training_x,training_y)
    RandomTree_classification=RanTree.predict(target_x)
    return RandomTree_classification


def SVM_classify(training_x, training_y,target_x,C_value):
    SVMachine=svm.SVC(kernel='rbf', C=C_value, gamma="scale")
    SVMachine.fit(training_x,training_y)
    SVM_classification=SVMachine.predict(target_x)
    return SVM_classification


def MLP_classify(training_x, training_y, target_x, hidden_layer):
    MLP_nn = MLPClassifier(hidden_layer_sizes=hidden_layer, activation='relu', solver='adam', max_iter=500)
    MLP_nn.fit(training_x,training_y)
    MLP_classification=MLP_nn.predict(target_x)
    return MLP_classification

In [ ]:
def CNN_classify(training_x, training_y, target_x, train_iteration,batch_size):
    training_x=tf.reshape(training_x,(training_x.shape[0], training_x.shape[1],1)) 
    target_x=tf.reshape(target_x,(target_x.shape[0], target_x.shape[1],1))
       
    model = keras.Sequential()
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(training_x.shape[1], training_x.shape[2])))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))  # 5-class output
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(training_x, training_y, epochs=train_iteration, batch_size=batch_size, verbose=0)

    result = model.predict(target_x)
    CNN_classification=np.argmax(result,axis=1)
    return CNN_classification
    #model implementation as guided by:
    #Brownlee, J. 2020. 1D Convolutional Neural Network Models for Human Activity Recognition. Available at: https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/ [Accessed: 27 April 2025]



def LSTM_classify(training_x,training_y,target_x,train_iteration,batch_size):
    training_x=tf.reshape(training_x,(training_x.shape[0], training_x.shape[1],1)) 
    target_x=tf.reshape(target_x,(target_x.shape[0], target_x.shape[1],1))
   
    model = keras.Sequential()
    model.add(layers.LSTM(units=32, activation='relu',input_shape=(training_x.shape[1], training_x.shape[2])))  
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

    model.fit(training_x, training_y, epochs=train_iteration, batch_size=batch_size, verbose=0)
    result = model.predict(target_x)
    LSTM_classification=np.argmax(result,axis=1)
    return LSTM_classification
    #model implementation as guided by:
    #n1k31t4, 2018. Keras LSTM with 1D time series. Data Science Stack Exchange. Available at: https://datascience.stackexchange.com/questions/27533/keras-lstm-with-1d-time-series [Accessed: 27 April 2025]

In [ ]:
#functions for finding the score from evaluation metrics function
def evaluate_performance(truth, predict,class_list, individually_bool):
    Performance_class={}
    predict=pd.Series(predict)

    acc=accuracy_score(y_true=truth,y_pred=predict)
    precision=precision_score(y_true=truth,y_pred=predict,average="weighted")
    recall=recall_score(y_true=truth,y_pred=predict,average="weighted",zero_division=0)
    f1Score=f1_score(y_true=truth, y_pred=predict,average="weighted",zero_division=0)
    recall_array=recall_score(y_true=truth,y_pred=predict,average=None,labels=class_list,zero_division=0)
    f1Score_array=f1_score(y_true=truth, y_pred=predict,average=None,labels=class_list,zero_division=0)
    if(individually_bool==True):
        overall_result=[acc,precision,recall,f1Score]
        for label in class_list:
            overall_result.append(recall_array[label])
            overall_result.append(f1Score_array[label])

        overall_result = pd.DataFrame([overall_result],
                                      columns=["accuracy_score","precision","recall","f1_score",
                                               "class_0_recall","class_0_f1_score",
                                               "class_1_recall","class_1_f1_score",
                                               "class_2_recall","class_2_f1_score",
                                               "class_3_recall","class_3_f1_score",
                                               "class_4_recall","class_4_f1_score"])

    else:
        overall_result={"accuracy":acc, "precision":precision,"recall":recall,"f1_score":f1Score}
        for label in class_list:
            Performance_class[label]={ "class_recall":recall_array[label],"class_f1_score":f1Score_array[label]}
        overall_result["individual"]=Performance_class
    return overall_result

# Result print out in text format
def final_printout(final_result):
    for window, info in final_result.items():
        print(f"from {window} seconds window:")
        print("-----------")
        for method, result in info.items():
            print(f"With {method} method the result is: ")
            for metric, data in result.items():
                if metric =="individual":
                    for individual, ind_result in data.items():
                        print(f"For {individual} class: {ind_result}")
                else:
                    print(f"{metric} is: {data}")
            print("++++++++++++++++++")
        print("==================")

In [ ]:
#function to execute the classifiers and collect the classification result
def classifier_collection(X_resample_train,Y_resample_train,X_target,Y_target,Signal_parameter,signal_source,class_list,hidden_layer,individually):
    Performance_stats={"KNN":{}, "NB":{}, "RF":{}, "SVM":{},"MLP":{},"CNN":{},"LSTM":{}}
    Performance_stats["KNN"].update(evaluate_performance(Y_target,KNN_classify(training_x=X_resample_train,training_y=Y_resample_train,target_x=X_target,k=Signal_parameter[signal_source]["k_number"]),class_list=class_list,individually_bool=individually))
    Performance_stats["NB"].update(evaluate_performance(Y_target,NaiveBayes_classify(training_x=X_resample_train,training_y=Y_resample_train,target_x=X_target),class_list=class_list,individually_bool=individually))
    Performance_stats["RF"].update(evaluate_performance(Y_target,RandomForest_classify(training_x=X_resample_train,training_y=Y_resample_train,target_x=X_target,tree_number=Signal_parameter[signal_source]["Tree_number"]),class_list=class_list,individually_bool=individually))
    Performance_stats["SVM"].update(evaluate_performance(Y_target,SVM_classify(training_x=X_resample_train,training_y=Y_resample_train,target_x=X_target,C_value=Signal_parameter[signal_source]["C_value"]),class_list=class_list,individually_bool=individually))
    Performance_stats["MLP"].update(evaluate_performance(Y_target,MLP_classify(training_x=X_resample_train,training_y=Y_resample_train,target_x=X_target,hidden_layer=hidden_layer),class_list=class_list,individually_bool=individually))
    Performance_stats["CNN"].update(evaluate_performance(Y_target,CNN_classify(training_x=X_resample_train,training_y=Y_resample_train,target_x=X_target,train_iteration=Signal_parameter[signal_source]["CNN_train_itera"], batch_size=Signal_parameter[signal_source]["CNN_batch_size"]),class_list=class_list,individually_bool=individually))
    Performance_stats["LSTM"].update(evaluate_performance(Y_target,LSTM_classify(training_x=X_resample_train,training_y=Y_resample_train,target_x=X_target,train_iteration=Signal_parameter[signal_source]["LSTM_train_itera"], batch_size=Signal_parameter[signal_source]["LSTM_batch_size"]),class_list=class_list,individually_bool=individually))
    return Performance_stats

In [ ]:
#Random Splitting Approach
def random_split_process(windows, signal_source, Signal_parameter,Testset_stage,individually):
    featureimportance_list=[]
    Final_Result={}
    for window in windows:
        print(window)

        #load and handle the data
        dataset,label,class_list=dataset_loading(window,signal_source,False)
        X_resample_train,Y_resample_train,X_valid,Y_valid,X_test,Y_test, importance_list= dataset_handling(dataset,label,Signal_parameter[signal_source]["PCA_target"],window)
        hidden_layer=Signal_parameter[signal_source]["hidden_layer"]
        
        featureimportance_list.append(importance_list)
        
        if Testset_stage==False:
            Final_Result[window]=classifier_collection(X_resample_train=X_resample_train,Y_resample_train=Y_resample_train,
                                                    X_target=X_valid,Y_target=Y_valid,Signal_parameter=Signal_parameter,
                                                    signal_source=signal_source,class_list=class_list,
                                                    hidden_layer=hidden_layer,individually=individually)
        else:
            Final_Result[window]=classifier_collection(X_resample_train=X_resample_train,Y_resample_train=Y_resample_train,
                                                   X_target=X_test,Y_target=Y_test,Signal_parameter=Signal_parameter,
                                                   signal_source=signal_source,class_list=class_list,
                                                   hidden_layer=hidden_layer,individually=individually)

    return Final_Result,featureimportance_list

In [ ]:
#Subject-wise Splitting Approach
def Subject_split_learning(windows, Signal_parameter, signal_source, test_stage):
    Final_Result={}
    for window in windows:
        print(window)
        #pre-handling 
        X_resample_train,Y_resample_train,X_valid,Y_valid,X_test,Y_test, Unique_class=individual_Load_handling(window=window,target_num=None,comp_num=Signal_parameter[signal_source]["PCA_target"],signal_source=signal_source,mode="subject")
        hidden_layer=Signal_parameter[signal_source]["hidden_layer"]

        if (test_stage):
          Final_Result[window]=classifier_collection(X_resample_train=X_resample_train,Y_resample_train=Y_resample_train,
                                                   X_target=X_test,Y_target=Y_test,Signal_parameter=Signal_parameter,
                                                   signal_source=signal_source,class_list=Unique_class,
                                                   hidden_layer=hidden_layer,individually=False)
        else:
          Final_Result[window]=classifier_collection(X_resample_train=X_resample_train,Y_resample_train=Y_resample_train,
                                                   X_target=X_valid,Y_target=Y_valid,Signal_parameter=Signal_parameter,
                                                   signal_source=signal_source,class_list=Unique_class,
                                                   hidden_layer=hidden_layer,individually=False)

    return Final_Result

In [ ]:
#Average Individual Approach
def individual_learning_process(windows, Signal_parameter,signal_source, individually, test_stage):
    Individual_result=pd.DataFrame()
    for window in windows:
        print(window)
        Individual_window={"KNN":[],"NB":[],"RF":[],"SVM":[],"MLP":[],"CNN":[],"LSTM":[]}
        for target_index in range(15):   #loop through every subject
            print(f"Processing {target_index}")
            #Load in and pre handle the data for learning
            X_resample_train,Y_resample_train,X_valid,Y_valid,X_test,Y_test, class_list=individual_Load_handling(window=window,target_num=target_index,comp_num=Signal_parameter[signal_source]["PCA_target"],signal_source=signal_source,mode="average")
            hidden_layer=Signal_parameter[signal_source]["hidden_layer"]
            
            if(test_stage):
              result=classifier_collection(X_resample_train=X_resample_train,Y_resample_train=Y_resample_train,
                                                    X_target=X_test,Y_target=Y_test,Signal_parameter=Signal_parameter,
                                                    signal_source=signal_source,class_list=class_list,
                                                    hidden_layer=hidden_layer,individually=individually)
            else:
              result=classifier_collection(X_resample_train=X_resample_train,Y_resample_train=Y_resample_train,
                                                    X_target=X_valid,Y_target=Y_valid,Signal_parameter=Signal_parameter,
                                                    signal_source=signal_source,class_list=class_list,
                                                    hidden_layer=hidden_layer,individually=individually)
            
            for key, value in result.items():
              if key in Individual_window:
                df=pd.DataFrame.from_dict(result[key]).T
                Individual_window[key].append(df)


        #merge and process the average result from every classifier
        for key in Individual_window.keys():
            Individual_window[key]=pd.concat(Individual_window[key],ignore_index=True,axis=1)
            Indi_mean=Individual_window[key].mean(axis=1)
            Indi_mean=Indi_mean.to_frame()
            Indi__classifier_mean=Indi_mean.rename(columns={0:f"{window}-{key}"})
            Individual_result=pd.concat([Individual_result,Indi__classifier_mean],axis=1)

    return Individual_result

In [ ]:
#Chest-Wrist Approach
def Chest_Wrist_learning(windows, Signal_parameter,individually, direction,test_stage):
    Final_Result={}
    signal_sources=list(Signal_parameter.keys())
    if direction==0:
        train_source,test_source=signal_sources[0],signal_sources[1]
        print("Trained by Wrist signal, Classify Chest Signal")
    else:
        train_source,test_source=signal_sources[1],signal_sources[0]
        print("Trained by Chest signal, Classify Wrist Signal")

    for window in windows:
        print(window)
        X_train,Y_train,train_unique=dataset_loading(window,train_source, True)
        X_target,Y_target,test_unique=dataset_loading(window,test_source, True)

        X_valid,X_test,Y_valid,Y_test=train_test_split(X_target,Y_target,  test_size=0.5, random_state=16,stratify=Y_target)

        X_resample_train,Y_resample_train,X_valid,X_test=standardise_smote(X_train,Y_train,X_valid,X_test)
        hidden_layer=Signal_parameter[train_source]["hidden_layer"]
        
        if(test_stage):
            Final_Result[window]=classifier_collection(X_resample_train=X_resample_train,Y_resample_train=Y_resample_train,
                                                    X_target=X_test,Y_target=Y_test,Signal_parameter=Signal_parameter,
                                                    signal_source=train_source,class_list=train_unique,
                                                    hidden_layer=hidden_layer,individually=individually)

        else:
            Final_Result[window]=classifier_collection(X_resample_train=X_resample_train,Y_resample_train=Y_resample_train,
                                                    X_target=X_valid,Y_target=Y_valid,Signal_parameter=Signal_parameter,
                                                    signal_source=train_source,class_list=train_unique,
                                                    hidden_layer=hidden_layer,individually=individually)
    return Final_Result

In [ ]:
#find average and top ranked feature
def feature_ranking(featureimportance_list):
    frame=pd.concat(featureimportance_list,axis=1)
    frame=frame.fillna(0)
    f_mean=frame.mean(axis=1)
    frame["Mean"]= f_mean
    final=frame.sort_values(by="Mean", ascending=False)
    final=final.rename(columns={0:"30 Second window",1:"60 Second window"})
    print("Combined importance of features from both windows")
    display(final)

In [ ]:
#Paremeter management for each approach and source
def parameter_control(subject_bool, average_bool, chest_wrist_bool):
  if(subject_bool):
    Signal_parameter={
    "Wrist":{"PCA_target":18,"k_number":31,"Tree_number":100,"C_value":1,"hidden_layer":(54,54,54),"CNN_train_itera":10,"CNN_batch_size":32,"LSTM_train_itera":15,"LSTM_batch_size":48},#
    "Chest":{"PCA_target":0.95,"k_number":31,"Tree_number":200,"C_value":1,"hidden_layer":(52,52,52),"CNN_train_itera":10,"CNN_batch_size":48, "LSTM_train_itera":10,"LSTM_batch_size":48},#
    "Combined":{"PCA_target":0.95,"k_number":31,"Tree_number":200,"C_value":1,"hidden_layer":(84,84,84),"CNN_train_itera":10,"CNN_batch_size":32,"LSTM_train_itera":15,"LSTM_batch_size":64}#
      }
  elif(average_bool):
    Signal_parameter={
      "Wrist":{"PCA_target":18,"k_number":21,"Tree_number":100,"C_value":1,"hidden_layer":(90,90,90),"CNN_train_itera":10,"CNN_batch_size":48,"LSTM_train_itera":15,"LSTM_batch_size":48},#
      "Chest":{"PCA_target":0.95,"k_number":21,"Tree_number":100,"C_value":1,"hidden_layer":(52,52,52),"CNN_train_itera":10,"CNN_batch_size":48, "LSTM_train_itera":15,"LSTM_batch_size":48},#
      "Combined":{"PCA_target":0.95,"k_number":15,"Tree_number":150,"C_value":1,"hidden_layer":(84,84,84),"CNN_train_itera":10,"CNN_batch_size":32,"LSTM_train_itera":15,"LSTM_batch_size":48}#
      }
  elif(chest_wrist_bool):
      Signal_parameter={
      "Wrist":{"PCA_target":None,"k_number":21,"Tree_number":150,"C_value":5,"hidden_layer":(54,54,54),"CNN_train_itera":10,"CNN_batch_size":32,"LSTM_train_itera":10,"LSTM_batch_size":32},
      "Chest":{"PCA_target":None,"k_number":21,"Tree_number":150,"C_value":1,"hidden_layer":(54,54,54),"CNN_train_itera":10,"CNN_batch_size":32, "LSTM_train_itera":10,"LSTM_batch_size":32}
      }
  else:#random
     Signal_parameter={
      "Wrist":{"PCA_target":18,"k_number":71,"Tree_number":100,"C_value":1,"hidden_layer":(54,54,54),"CNN_train_itera":10,"CNN_batch_size":32,"LSTM_train_itera":18,"LSTM_batch_size":32},
      "Chest":{"PCA_target":0.95,"k_number":71,"Tree_number":100,"C_value":1,"hidden_layer":(52,52,52),"CNN_train_itera":10,"CNN_batch_size":32, "LSTM_train_itera":15,"LSTM_batch_size":32},
      "Combined":{"PCA_target":0.95,"k_number":71,"Tree_number":100,"C_value":1,"hidden_layer":(84,84,84),"CNN_train_itera":10,"CNN_batch_size":32,"LSTM_train_itera":15,"LSTM_batch_size":32}
      }
  return Signal_parameter

In [ ]:
#main flow: call the classify function and evaluate function

signal_source="Chest" #"Combined"#"Wrist"
if signal_source =="Combined":
  windows=[60]
else:
  windows=[30,60] # 30 or 60


#Boolean to control the apporach for execution
Testset_stage=False
subject_split=False
average_individually=False
Chest_Wrist_Mix=False
Chest_wrist_direction=0 #0= Wrist train -> Chest test   #1=Chest train -> Wrist test

#select corresponding parameter
Signal_parameter=parameter_control(subject_split,average_individually,Chest_Wrist_Mix)

#Approach
if(average_individually):#one out at a time and average it
    Individual_result=individual_learning_process(windows,Signal_parameter,signal_source,average_individually,Testset_stage)
elif(subject_split):#subject-wise splitting
    Final_Result=Subject_split_learning(windows, Signal_parameter,signal_source,Testset_stage)
elif(Chest_Wrist_Mix):#Chest or Wrist classify the other
    Final_Result=Chest_Wrist_learning(windows, Signal_parameter,False,Chest_wrist_direction,Testset_stage)
else:#random split from the merged
    Final_Result,featureimportance_list=random_split_process(windows, signal_source, Signal_parameter,Testset_stage,individually=False)


#result
print(f"signal source: {signal_source}")
if (average_individually):
    print("Average individually")
    #directly displaying the dataframe
    display(Individual_result)
else:
    #result presentation
    if( Testset_stage==True):
        print("Test Set")
    else:
        print("Valid Set")
    if(Chest_Wrist_Mix):
        print("Chest Wrist set")
    elif(subject_split):
        print("Split by num of Subject")
    else:
        print("Random Split")
    final_printout(Final_Result)
    #feature_ranking(featureimportance_list)